In [9]:
import numpy as np 
import pandas 
import re
import nltk
import wordcloud
import numpy as np 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from langdetect import detect
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from datetime import datetime as dt
import seaborn as sb
from os import path
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df = pandas.read_csv('/Users/hejielu/Desktop/metoo.csv')
df.head()

,Created_at,Tweet_Content
0,6/3/20 16:16,RT @BlackTrump16: Israeli Priviledge. Matt lau...
1,6/3/20 16:15,Not my normal viewing but this documentary on ...
2,6/3/20 16:15,wish can say the story ；need more voice and su...
3,6/3/20 16:15,"@TheSandraG psst, I saw your bit about Blackou..."
4,6/3/20 16:15,RT @ms_rivera91: It hurts my heart to see for ...


In [11]:
def preprocess(df):
    for d in df.itertuples():
        for t in df.itertuples():
            
            #format the 'dateoftweet' column value into mm/dd/yyyy
            if len(str(d[1])) > 20:
                temp_month = str(d[1])[4:7]
                date = str(d[1])[8:10]
                year = str(d[1])[26:30]
                months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sept': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
                month = ''
                for key,value in months.items():
                    if temp_month == key:
                        month = value
                dt = str(month) + '/' + date + '/' + year
            else:
                dt = str(d[1])[5:7] + '/' + str(d[1])[8:10] + '/' + str(d[1])[0:4]
            df.at[d.Index, 'dateoftweet'] = dt
            
            #use regular expression in 'text' column
            txt = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(t[2]))
            p = re.compile(r'\<http.+?\>', re.DOTALL)
            txt = re.sub(p, '', txt)
            txt = txt.lower()
            txt = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", txt)
            txt = re.sub("(\\d|\\W)+"," ",txt)
            txt = re.sub('rt', '', txt)
            txt = re.sub('amp', '', txt)
            txt = txt.split()
            lemm = WordNetLemmatizer()
            txt = [lemm.lemmatize(word) for word in txt if not word in STOPWORDS] 
            txt = " ".join(txt)
            df.at[t.Index, 'text'] = txt
    return df

In [12]:
df = preprocess(df)

In [13]:
df.to_csv('/Users/hejielu/Desktop/dta.csv', index=False, encoding='utf-8')

In [26]:
df1 = pandas.read_csv('/Users/hejielu/Desktop/final/cleandata.csv')
df1.head()

,dateoftweet,text
0,03/05/2020,suck everyday don t security people infiltrate...
1,03/05/2020,s democrat front runner tonight metoo activist...
2,03/05/2020,housing training pay aggrieved female resident...
3,03/05/2020,gaining traction metoo movement victim patel s...
4,03/05/2020,onair chill take midol democrat pollster kil


In [27]:
df2 = pandas.read_csv('/Users/hejielu/Desktop/metoo.csv')
df2.head()

,dateoftweet,text
0,06/03/2020,israeli priviledge matt lauer somehow jail bac...
1,06/03/2020,normal viewing documentary netflix essential s...
2,06/03/2020,wish say story need voice suppo metoo
3,06/03/2020,psst saw bit blackout tuesday point previous t...
4,06/03/2020,rivera hus hea see many former female student ...


In [28]:
df_total = pandas.concat([df2, df1],ignore_index=True)

In [30]:
df_total.describe

<bound method NDFrame.describe of       dateoftweet                                               text
0      06/03/2020  israeli priviledge matt lauer somehow jail bac...
1      06/03/2020  normal viewing documentary netflix essential s...
2      06/03/2020              wish say story need voice suppo metoo
3      06/03/2020  psst saw bit blackout tuesday point previous t...
4      06/03/2020  rivera hus hea see many former female student ...
5      06/03/2020            uh dont feel ashamed still shameful act
6      06/03/2020                                              metoo
7      06/03/2020  see parallel metoo movement suppo tara reade g...
8      06/03/2020  s fair question first thing d say dynamic soci...
9      06/03/2020  lmaoo laughable seen powerful men essentially ...
10     06/03/2020   ahhhhlexis seeing hea dropped conversation metoo
11     06/03/2020                                                NaN
12     06/03/2020  people shouting correction helping either meto...


In [31]:
df_total.replace('', np.nan, inplace=True)

In [32]:
df_total.dropna(how='all', subset=['text'], inplace=True)

In [33]:
df_total.drop_duplicates(subset='text', keep="first")

,dateoftweet,text
0,06/03/2020,israeli priviledge matt lauer somehow jail bac...
1,06/03/2020,normal viewing documentary netflix essential s...
2,06/03/2020,wish say story need voice suppo metoo
3,06/03/2020,psst saw bit blackout tuesday point previous t...
4,06/03/2020,rivera hus hea see many former female student ...
5,06/03/2020,uh dont feel ashamed still shameful act
6,06/03/2020,metoo
7,06/03/2020,see parallel metoo movement suppo tara reade g...
8,06/03/2020,s fair question first thing d say dynamic soci...
9,06/03/2020,lmaoo laughable seen powerful men essentially ...


In [34]:
df_total.to_csv('/Users/hejielu/Desktop/cleandata.csv', index=False, encoding='utf-8')